In [12]:
import requests
import pandas as pd

# Define the endpoint URL
endpoint_url = "https://financialmodelingprep.com/api/v3/historical-price-full/AAPL?apikey=wc2bbHWhFBL7no45kaUlx2xLHI2z2wv1"

# Fetch data from the endpoint
response = requests.get(endpoint_url)
data = response.json()

# Check if data is fetched successfully
if "historical" in data:
    # Convert data to DataFrame
    df = pd.DataFrame(data["historical"])
    
    # Display the first few rows of the DataFrame
    print("Sample data from the API endpoint:")
    print(df.head())
    
    # Perform initial data analysis or visualization as needed
    
else:
    print("Failed to fetch data from the API endpoint.")


ModuleNotFoundError: No module named 'requests'